<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('/datasets/insurance.csv')

df.isna().sum()
df.describe()
df.duplicated().sum()

в данных нет пропусков, зато есть 153 дубликата. удалять ли их?


In [ ]:
df.info()
df['Возраст'].unique()
df['Зарплата'].unique()

Дробных значений в столбцах типа float нет. Изменим их тип на int.

In [ ]:
df['Возраст'] = df['Возраст'].astype('int')
df['Зарплата'] = df['Зарплата'].astype('int')
df.info()

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится.

**Обоснование:** 
Заменим матрицу признаков X на матрицу F:

$$
F = XP    \qquad (2.1)
$$

где P - обратимая матрица, на которую может быть умножена матрица X.

Теперь вычислим, чему будет равно предсказание и вектор весов регрессии.
$$
a_1 = Fw_1 \qquad (2.2)
$$

$$
w_1 = (F^T F)^{-1} F^T y \qquad (2.3)
$$

Подставим в уравнение 2.2 правую часть уравнения 2.3. Получим следующее:

$$
a_1 = F (F^T F)^{-1} F^T y \qquad (2.4)
$$

Заменим все F на правую часть уравнения 2.1:

$$a_1 = XP ((XP)^T XP)^{-1} (XP)^T y \qquad (2.5)
$$

Раскроем уравнение 2.5, применив правило транспонирования матриц: транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке.
$$
a_1 = XP(P^T X^T XP)^{-1} P^T X^T y \qquad (2.6.1)
$$

Теперь воспользуемся свойством сочетания матриц: обратная матрица произведения равна произведению обратных матриц, взятых в обратном порядке:

$$
a_1 = XPP^{-1} (P^T X^T X)^{-1} P^T X^T y \qquad (2.7.1)
$$
$$
a_1 = XPP^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y \qquad (2.7.2)
$$

Наконец, воспользуемся свойством, гласящим, что при умножении матрицы на обратную матрицу получается единичная матрица E. Так, сократим уравнение 2.7.2:

$$
a_1 = X(X^T X)^{-1} X^T y  \qquad (2.8.1)
$$

Тем самым, мы пришли к той же формуле, что $ a = Xw $ и можем утверждать, что $ a_1 = a $

Таким образом, мы доказали, что при умножении матрицы признаков на обратную матрицу качество предсказания a не изменится.

__Какое соотношение между 𝑤 и 𝑤𝑝?__

Если $a_1 = Fw_1 \qquad (2.2)$,

a, как мы доказали выше, 
 
$a_1 = a \qquad (2.8)$, 

то соотношение между 𝑤 и 𝑤𝑝 можно признать равным.


**Алгоритм**

Для защиты информации клиентов страховой компании можно умножать матрицу признаков линейной регрессии на обратимую матрицу Z, генерируемую случайным образом.

Для этого потребуются следующие этапы:
1) Составление матрицы Z


2) Проверка этой матрицы на обратимость и вычисление детерминанта


3) Получение преобразованной матрицы признаков: Y = XZ


4) Применение алгоритма линейной регрессии на преобразованных признаках Y

**Обоснование**

Умножение матрицы на матрицу возможно, если ширина первой матрицы А (𝑚 × 𝑛) равна высоте второй матрицы В (𝑛 × r). То есть наша матрица Z должна иметь размерность (𝑚 × 𝑛), где n соответствует количеству признаков регрессии. А матрица Y будет иметь такую же размерность, что и матрица X.

$$
X = \begin{pmatrix}
1 & 2 \\
2 & 3 \\
4 & 5  
\end{pmatrix}
\qquad 
Z = \begin{pmatrix}
1 & 0 \\
2 & 3 \\  
\end{pmatrix}
\qquad det  Y = 3
$$

Найдем Y:

$$
Y = \begin{pmatrix}
1 & 2 \\
2 & 3 \\
4 & 5  
\end{pmatrix} \begin{pmatrix}
1 & 0 \\
2 & 3 \\  
\end{pmatrix} = \begin{pmatrix}
1*1+2*2 & 1*0+2*3  \\
2*1+3*2 & 2*0+3*3  \\
4*1+5*2 & 4*0+5*3   
\end{pmatrix} = \begin{pmatrix}
5 & 6 \\
8 & 9 \\
14 & 15  
\end{pmatrix}
$$

После этого необходимо добавить нулевой столбец, после чего матрица готова для использования в линейной регрессии.

## Проверка алгоритма

Проведем исследование модели по двум направлениям:

Сначала исследуем качество модели без преобразования. 1.1 С исходными признаками 1.2 С отмасштабированными признаками

Затем исследуем качество модели с преобразованием. 2.1 С исходными признаками 2.2 С отмасштабированными признаками

Разделим данные на обучающие и тестовые выборки.

In [ ]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_initial_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef",model.coef_)
print("R2 =", R2_LR_initial_data)


In [ ]:
regression = LinearRegression()
scaler = StandardScaler()
pipeline = Pipeline([("standard_scaler", scaler),("linear_regression", regression)])
pipeline.fit(features_train, target_train)
R2_LR_scaled_data = r2_score(target_test, pipeline.predict(features_test))
#print("w-vector coef",pipeline.coef_) при использовании pipeline невозможно получить коэффициенты регрессии
print("R2 =", R2_LR_scaled_data)

Видим, что коэффициент R2 в обоих случаях равный, как на исходных данных, так и на скалированных. 

Приступим к преобразованию матрицы признаков.

In [ ]:
def cipher(features):
    crypted_features = features
    n = features.shape[1]
    np.random.seed(12345)
    cipher_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(cipher_matrix)
    while det == 0:
        cipher_matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(cipher_matrix)
    crypted_features = crypted_features @ cipher_matrix
    return crypted_features, cipher_matrix

Посмотрим на данные до преобразования и после.

In [ ]:
display(features.head())
features, cipher_matrix = cipher(features)
display(features.head())
cipher_matrix

Мы видим, что данные благополучно зашифрованы, вместо оригинальных значений выводятся какие-то непонятные числа. 
Разобьем их теперь на тренировочную и обучающую выборки и проверим значение R2 на зашифрованных данных.

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)


In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_cipher_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef",model.coef_)
print("R2 =", R2_LR_cipher_data)


In [ ]:
regression = LinearRegression()
scaler = StandardScaler()
pipeline = Pipeline([("standard_scaler", scaler),("linear_regression", regression)])
pipeline.fit(features_train, target_train)
R2_LR_cipher_data = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_cipher_data)

Показатель R2 не изменился. ЧТД.

## Выводы

В ходе работы с данными страховой компании "Хоть потом" мы проделали следующее: 

1. Загрузили и изучили исходные данные.
2. Доказали, что качество линейной регресии не меняется при использования исходной матрицы или исходной матрицы, умноженной на обратимую.
3. Создали алгоритм шифрования данных.
4. Исследовали его работу с проверкой значений метрики R2 для исходных данных и для шифрованных.

Результат: данные клиентов можно нетрудным путем зашифровать, и это никак не повлияет на значения линейной регрессии.